In [1]:
!pip3 install --upgrade pip --user
!pip3 install kfp --user

In [2]:
import kfp
from kfp import *
import kfp.components as comp
import kfp.dsl as dsl
from kfp import gcp
import os
from pathlib import Path
import tensorflow as tf


In [3]:
def convert_doccano_fomart_to_spacy(data_path):
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'spacy==2.0.18'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'spacy download en'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'gcsfs'])
    import json
    import numpy as np
    import gcsfs
    import plac
    import random
    import warnings
    from pathlib import Path
    import spacy

    #read from gcs bucket
    fs = gcsfs.GCSFileSystem(project='mlops-kubeflow-00')
    with fs.open('gs://nerdoc/training_data.json1', 'rb') as f:
        data = f.readlines()
        print(data)
        

    training_data = []
    for record in data:
        entities = []
        read_record = json.loads(record)
        text = read_record['text']
        entities_record = read_record['labels']

        for start, end, label in entities_record:
            entities.append((start, end, label))

        training_data.append((text, {"entities": entities}))

    with open(f'{data_path}/training_data.txt', 'w') as f:
        json.dump(training_data, f)

    print('dat saved successfully')

In [4]:
def train(data_path, iterations = 2):
    def train_spacy(data_path, iterations = 2):
        import sys, subprocess;
        subprocess.run([sys.executable, '-m', 'pip', 'install', 'spacy==2.0.18'])
        subprocess.run([sys.executable, '-m', 'pip', 'install', 'spacy download en'])
        subprocess.run([sys.executable, '-m', 'pip', 'install', 'gcsfs'])
        #Converting JSON1 file to Spacy tuples format
        import json
        import numpy as np
        import plac
        import gcsfs
        import random
        import warnings
        from pathlib import Path
        import spacy
        import logging
        from spacy.util import minibatch, compounding
        from spacy.gold import GoldParse
        from spacy.scorer import Scorer

        with open(f'{data_path}/training_data.txt', 'r') as f:
                TRAIN_DATA = json.load(f)
                print(TRAIN_DATA)
        nlp = spacy.blank('en') 
        if 'ner' not in nlp.pipe_names:
            ner = nlp.create_pipe('ner')
            nlp.add_pipe(ner, last=True)


        for _, annotations in TRAIN_DATA:
             for ent in annotations.get('entities'):
                ner.add_label(ent[2])

        other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
        with nlp.disable_pipes(*other_pipes): 
            optimizer = nlp.begin_training()
            for itn in range(iterations):
                print("Starting iteration " + str(itn))
                random.shuffle(TRAIN_DATA)
                losses = {}
                for text, annotations in TRAIN_DATA:
                    try:
                        nlp.update(
                            [text],  
                            [annotations],  
                            drop=0.2,  
                            sgd=optimizer,  
                            losses=losses)
                    except Exception as error:
                        print(error)
                        continue
                print(losses)
        return nlp


    trainer = train_spacy(data_path, iterations=2)
        # Save our trained Model
    
    trainer.to_disk(f'{data_path}/ner_model')

    print('Model saved successfully')

In [5]:
def test_spacy(data_path):
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'spacy==2.0.18'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'spacy download en'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'gcsfs'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn'])
    import spacy
    from spacy.gold import GoldParse
    from spacy.scorer import Scorer
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import precision_recall_fscore_support
    import argparse
    import os
    import json
    import numpy as np
    import plac
    import random
    import warnings
    from pathlib import Path
    import spacy
    import gcsfs
        
    def convert_doccano_fomart_to_spacy(inputs):
   # read test data from GCS bucket
        fs = gcsfs.GCSFileSystem(project='mlops-kubeflow-00')
        with fs.open(inputs, 'rb') as f:
            data = f.readlines()
            print(data)

        training_data = []
        for record in data:
            entities = []
            read_record = json.loads(record)
            text = read_record['text']
            entities_record = read_record['labels']

            for start, end, label in entities_record:
                entities.append((start, end, label))

            training_data.append((text, {"entities": entities}))

        return training_data
    
    test = 'gs://nerdoc/test_data.json1'
    test = convert_doccano_fomart_to_spacy(test)
    nlp = spacy.load(f'{data_path}/ner_model')
    print('model loaded')
    
        
        
    #test the model and evaluate it
    examples = test
    tp=0
    tr=0
    tf=0

    ta=0
    c=0        

    for text,annot in examples:
        fs = gcsfs.GCSFileSystem(project='mlops-kubeflow-00')
        f=fs.open(os.path.join(f'{data_path}/test'+"result"+str(c)+".txt"), 'w')
        doc_to_test=nlp(text)
        d={}
        for ent in doc_to_test.ents:
            d[ent.label_]=[]
        for ent in doc_to_test.ents:
            d[ent.label_].append(ent.text)

        for i in set(d.keys()):

            f.write("\n\n")
            f.write(i +":"+"\n")
            for j in set(d[i]):
                f.write(j.replace('\n','')+"\n")
        d={}
        for ent in doc_to_test.ents:
            d[ent.label_]=[0,0,0,0,0,0]
        for ent in doc_to_test.ents:
            doc_gold_text= nlp.make_doc(text)
            gold = GoldParse(doc_gold_text, entities=annot.get("entities"))
            y_true = [ent.label_ if ent.label_ in x else 'Not '+ent.label_ for x in gold.ner]
            y_pred = [x.ent_type_ if x.ent_type_ ==ent.label_ else 'Not '+ent.label_ for x in doc_to_test]  
            if(d[ent.label_][0]==0):
                (p,r,f,s)= precision_recall_fscore_support(y_true,y_pred,average='weighted')
                a=accuracy_score(y_true,y_pred)
                d[ent.label_][0]=1
                d[ent.label_][1]+=p
                d[ent.label_][2]+=r
                d[ent.label_][3]+=f
                d[ent.label_][4]+=a
                d[ent.label_][5]+=1
        c+=1
    for i in d:
        print("\n For Entity "+i+"\n")
        print("Accuracy : "+str((d[i][4]/d[i][5])*100)+"%")
        print("Precision : "+str(d[i][1]/d[i][5]))
        print("Recall : "+str(d[i][2]/d[i][5]))
        print("F-score : "+str(d[i][3]/d[i][5]))


    print('Test completed')

In [6]:
# Create train and predict lightweight components.
prep_op = comp.func_to_container_op(convert_doccano_fomart_to_spacy, base_image="python:3.7")
train_op = comp.func_to_container_op(train, base_image="python:3.7")
test_op = comp.func_to_container_op(test_spacy, base_image="python:3.7")

/home/jovyan/.local/lib/python3.6/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "Integer" based on the value "2".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))


In [7]:
client = kfp.Client()

In [8]:
# Define the pipeline
@dsl.pipeline(
   name='ner pipeline',
   description=''
)

# Define parameters to be fed into pipeline
def ner_container_pipeline(
    data_path: str,
    iterations: int, 
):
    
    # Define volume to share data between components.
    vop = dsl.VolumeOp(
    name="create_volume",
    resource_name="data-volume", 
    size="1Gi", 
    modes=dsl.VOLUME_MODE_RWO)
    
    # Create NER training component.
    prep_container = prep_op(data_path) \
                                    .add_pvolumes({data_path: vop.volume})

    # Create NER training component.
    train_container = train_op(data_path, iterations) \
                                    .add_pvolumes({data_path: prep_container.pvolume})
    
    # Create NER testing component.
    test_container = test_op(data_path) \
                                    .add_pvolumes({data_path: train_container.pvolume})

In [9]:
DATA_PATH = '/mnt'
ITERATIONS=2


In [ ]:
pipeline_func = ner_container_pipeline
experiment_name = 'ner_kubeflow'
run_name = pipeline_func.__name__ + ' run'

arguments = {"data_path":DATA_PATH,
             "iterations":ITERATIONS}

# Compile pipeline to generate compressed YAML definition of the pipeline.
kfp.compiler.Compiler().compile(pipeline_func,  
  '{}.zip'.format(experiment_name))

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)